## **Imports**

In [1]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
from application.modeling import balance_sheet
from application.modeling import statement_of_cashflows
from application.modeling import loan_book
from application.modeling import borrowings

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [2]:
start_date = "2024-01"
months_to_forecast = 60
imtt = 0.01

In [3]:
importlib.reload(helper)

<module 'application.modeling.helper' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\helper.py'>

In [4]:
disbursement_parameters = pd.read_csv("./data_templates/disbursement_parameters.csv", index_col=0)
disbursement_parameters = helper.match_months_to_forecast_with_df(
    df=disbursement_parameters,
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)
disbursement_parameters.head()

,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,2027-01,2027-02,2027-03,2027-04,2027-05,2027-06,2027-07,2027-08,2027-09,2027-10,2027-11,2027-12,2028-01,2028-02,2028-03,2028-04,2028-05,2028-06,2028-07,2028-08,2028-09,2028-10,2028-11,2028-12
AGENT_COMMISSION,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
AGENT_CONTRIBUTION_PERCENT,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
BUSINESS_ACQUISITION_PERCENT,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
CREDIT_OFFICER_COMMISSION,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
CREDIT_OFFICER_SALARY,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00


In [5]:
importlib.reload(helper)

<module 'application.modeling.helper' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\helper.py'>

In [6]:
other_parameters = pd.read_csv("./data_templates/other_parameters.csv", index_col=0)
other_parameters = helper.match_months_to_forecast_with_df(
    df=other_parameters,
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)
other_parameters

,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,2027-01,2027-02,2027-03,2027-04,2027-05,2027-06,2027-07,2027-08,2027-09,2027-10,2027-11,2027-12,2028-01,2028-02,2028-03,2028-04,2028-05,2028-06,2028-07,2028-08,2028-09,2028-10,2028-11,2028-12
NEW_INVENTORY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
INVENTORY_USED,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_INTERGROUP_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_TRADE_PAYABLES,0.00,0.00,0.00,0.00,0.00,"15,888.40","19,217.95","32,088.24","3,425.38","5,042.59","5,990.38","10,202.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_TRADE_RECEIVABLES,0.00,"91,225.40","57,639.36","58,279.67",0.00,"27,773.92","1,728.17","2,654.09","3,425.38","5,042.59","5,990.38","10,202.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
PAYMENTS_TO_OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
PAYMENTS_TO_TRADE_PAYABLES,0.00,"13,404.72","68,786.66","80,963.15","466,687.36",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
RECEIPTS_FROM_TRADE_RECEIVABLES,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
details_of_assets = pd.read_csv("./data_templates/details_of_assets.csv")
details_of_assets.head()

,Asset ID,Acquisition Date,Life,Book Value,Net Value,Salvage Value,Method,Depreciation
0,FF-HRE-0024,01/09/2015,10,380.00,75.96,0,straight_line,NaN
1,FF-HRE-0025,01/09/2015,10,370.00,74.04,0,straight_line,NaN
2,FF-HRE-0026,01/09/2015,10,370.00,74.04,0,straight_line,NaN
3,FF-HRE-0027,01/09/2015,10,630.00,126.00,0,straight_line,NaN
4,FF-HRE-0028,01/09/2015,10,630.00,126.00,0,straight_line,NaN


In [8]:
details_of_long_term_borrowing = pd.read_csv("./data_templates/details_of_long_term_borrowing.csv")
details_of_long_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,31/12/2022,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2022,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance
2,31/12/2022,Grandre,160000,0.06,36,Quaterly (I+Bullet P),4,straight_line
3,31/12/2022,MINERVA,15000,0.08,36,Bullet (P+I),0,straight_line
4,10/09/2022,ZAM-CARGO,364000,0.05,36,Annually(I) Principal rollover,12,straight_line


In [9]:
details_of_short_term_borrowing = pd.read_csv("./data_templates/details_of_short_term_borrowing.csv")
details_of_short_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,24/04/2023,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,02/03/2023,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,31/12/2022,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,31/12/2022,Grandre,"100,000.00",0.12,12,Quaterly (I+Bullet P),4,straight_line
4,23/04/2023,FCB,"18,429.93",1.50,12,Bullet (P+I),0,straight_line


In [10]:
expenses_certain = pd.read_csv("./data_templates/expenses_certain.csv", index_col=0)

expenses_certain = helper.match_months_to_forecast_with_df(
    df=expenses_certain,
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

expenses_certain.head()

,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,2027-01,2027-02,2027-03,2027-04,2027-05,2027-06,2027-07,2027-08,2027-09,2027-10,2027-11,2027-12,2028-01,2028-02,2028-03,2028-04,2028-05,2028-06,2028-07,2028-08,2028-09,2028-10,2028-11,2028-12
Training,40.00,40.00,"6,840.00","1,740.00","1,540.00","2,540.00","1,540.00",40.00,"2,640.00",40.00,40.00,40.00,"1,493.67","1,498.65","1,503.65","1,508.66","1,513.69","1,518.73","1,523.80","1,528.87","1,533.97","1,539.08","1,544.21","1,548.07","1,551.94","1,555.82","1,559.71","1,563.61","1,567.52","1,571.44","1,575.37","1,579.31","1,583.26","1,587.21","1,591.18","1,595.16","1,599.15","1,603.15","1,607.15","1,611.17","1,615.20","1,619.24","1,623.29","1,627.34","1,631.41","1,635.49","1,639.58","1,643.68","1,647.79","1,651.91","1,656.04","1,660.18","1,664.33","1,668.49","1,672.66","1,676.84","1,681.03","1,685.24","1,689.45","1,693.67"
CILL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Provision,"2,023.00","2,023.00","2,023.00","2,160.50","2,577.17","2,577.17","2,906.33","3,056.33","3,056.33","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,189.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,339.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,539.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,689.67","3,739.67"
Retrenchments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Staff Welfare,"17,275.00","17,275.00","17,275.00","17,275.00","17,275.00","17,706.88","17,706.88","17,706.88","17,706.88","17,706.88","17,706.88","18,052.38","18,052.38","18,052.38","18,052.38","18,052.38","18,052.38","18,397.88","18,397.88","18,397.88","18,397.88","18,397.88","18,397.88","18,657.00","18,657.00","18,657.00","18,657.00","18,657.00","18,657.00","18,916.12","18,916.12","18,916.12","18,916.12","18,916.12","18,916.12","19,175.25","19,175.25","19,175.25","19,175.25","19,175.25","19,175.25","19,434.38","19,434.38","19,434.38","19,434.38","19,434.38","19,434.38","19,693.50","19,693.50","19,693.50","19,693.50","19,693.50","19,693.50","19,952.62","19,952.62","19,952.62","19,952.62","19,952.62","19,952.62","20,211.75"


In [11]:
expenses_uncertain = pd.read_csv("./data_templates/expenses_uncertain.csv", index_col=0)
expenses_uncertain = helper.match_months_to_forecast_with_df(
    df=expenses_uncertain,
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)
expenses_uncertain.head()

,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,2027-01,2027-02,2027-03,2027-04,2027-05,2027-06,2027-07,2027-08,2027-09,2027-10,2027-11,2027-12,2028-01,2028-02,2028-03,2028-04,2028-05,2028-06,2028-07,2028-08,2028-09,2028-10,2028-11,2028-12
1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [12]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS,INVESTMENT_IN_SUBSIDIARIES,INVESTMENT_IN_ASSOCIATES,INVESTMENT_PROPERTIES,EQUITY_INVESTMENTS,LONG_TERM_MONEY_MARKET_INVESTMENTS,LOANS_TO_RELATED_ENTITIES,TREASURY_SHARES.1,INTANGIBLE_ASSETS,INVENTORIES,OTHER_RECEIVABLES.1,INTERGROUP_RECEIVABLES,SHORT_TERM_MONEY_MARKET_INVESTMENTS
0,1092458,938595,2806038,331856,7050,110,59784,0,876511,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv")
existing_loans.head()

,Loan Number,Disbursement Date,Loan Term,Loan Amount,Closing Balance,Interest Rate,Admin Fee,Credit Insurance Fee,Loan Type
0,"100,120,644,000,001",13/10/2021,60,"20,000.00","-14,769.50",0.00,0.02,0.02,B2B
1,"100,120,644,000,002",15/10/2021,60,"12,000.00","-8,855.52",0.00,0.02,0.02,B2B
2,"100,120,644,000,003",25/10/2021,60,"20,000.00","-14,708.39",0.00,0.02,0.02,B2B
3,"100,120,644,000,004",26/10/2021,60,"20,000.00","-14,703.34",0.00,0.02,0.02,B2B
4,"100,120,644,000,005",28/10/2021,60,"20,000.00","-14,693.08",0.00,0.02,0.02,B2B


## **New Disbursements**

In [14]:
new_disbursements_df  = disbursements.calculate_new_disbursements(disbursement_parameters=disbursement_parameters)
new_disbursements_df

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2024-01,"57,011.05","360,000.00","48,000.00","96,000.00","609,011.05"
2024-02,"27,480.25","384,000.00","60,000.00","120,000.00","651,480.25"
2024-03,"27,480.25","432,000.00","72,000.00","172,800.00","805,080.25"
2024-04,"34,900.00","342,000.00","80,000.00","216,000.00","808,900.00"
2024-05,"66,173.00","513,000.00","96,000.00","259,200.00","1,097,573.00"
2024-06,"18,709.25","540,000.00","96,000.00","291,600.00","1,141,909.25"
2024-07,"40,650.00","600,000.00","168,000.00","378,000.00","1,396,650.00"
2024-08,"8,680.00","770,000.00","180,000.00","405,000.00","1,588,680.00"
2024-09,"65,000.00","770,000.00","180,000.00","450,000.00","1,735,000.00"
2024-10,"70,000.00","1,125,000.00","180,000.00","450,000.00","2,095,000.00"


### **Calculate Repayment Amount For New Disbursements**

In [15]:
monthly_repayment_new_disbursements_df = (
    interest_income.calculate_monthly_repayments_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
    )
)
monthly_repayment_new_disbursements_df.head()

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment,total
2024-01,"81,451.18","5,779.79","8,006.34","16,012.68","111,250.00"
2024-02,"86,881.26","2,785.95","10,007.93","20,015.86","119,691.00"
2024-03,"97,741.42","2,785.95","12,009.51","28,822.83","141,359.72"
2024-04,"77,378.62","3,538.17","13,343.90","36,028.54","130,289.24"
2024-05,"116,067.93","6,708.63","16,012.68","43,234.25","182,023.50"


### **Generate Loan Schedules For New Disbursements**

In [16]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=months_to_forecast,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

In [17]:
loan_schedules_for_all_new_disbursements['sme_loan_schedules'].keys()

dict_keys(['interest', 'capital_repayments', 'outstanding_at_start'])

### **Capital Repayment New Disbursement**

In [18]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)
capital_repayment_new_disbursements_df.head()

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2024,"47,251.18","5,637.26","7,372.68","3,686.34","63,947.47"
Mar-2024,"102,141.30","8,368.61","17,252.08","8,626.04","136,388.03"
Apr-2024,"168,546.15","11,106.78","32,075.60","14,931.90","226,660.43"
May-2024,"229,446.65","14,585.46","51,550.95","22,419.67","318,002.74"
Jun-2024,"318,577.02","21,165.12","76,096.78","31,810.13","447,649.05"


### **Interest Income New Disbursements**


In [19]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)

interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2024,"34,200.00",142.53,"8,640.00","4,320.00","47,302.53"
Mar-2024,"66,191.14",197.14,"18,776.46","9,388.23","94,552.96"
Apr-2024,"97,527.71",244.91,"32,775.77","15,091.89","145,640.28"
May-2024,"114,005.83",304.40,"49,328.97","20,948.01","184,587.21"
Jun-2024,"140,943.40",433.37,"68,017.38","27,570.24","236,964.39"
Jul-2024,"161,978.58",427.23,"87,412.67","33,347.33","283,165.81"
Aug-2024,"179,105.31",471.18,"111,952.07","44,683.22","336,211.78"
Sep-2024,"208,850.51",425.02,"136,379.10","56,007.81","401,662.44"
Oct-2024,"233,124.79",517.34,"161,964.29","66,089.28","461,695.71"
Nov-2024,"286,165.14",605.92,"184,196.30","74,815.74","545,783.10"


### **Calculate Admin Fee On New Disbursements**

In [20]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    disbursement_parameters=disbursement_parameters,
    months_to_forecast=months_to_forecast,
)

admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2024,"3,000.00",114.02,218.18,436.36,"3,768.57"
Feb-2024,"6,200.00",168.98,490.91,981.82,"7,841.71"
Mar-2024,"9,800.00",223.94,818.18,"1,767.27","12,609.40"
Apr-2024,"12,650.00",293.74,"1,181.82","2,749.09","16,874.65"
May-2024,"16,925.00",426.09,"1,618.18","3,927.27","22,896.54"
Jun-2024,"21,425.00",463.51,"2,054.55","5,252.73","29,195.78"
Jul-2024,"23,425.00",544.81,"2,818.18","6,970.91","33,758.90"
Aug-2024,"26,641.67",562.17,"3,636.36","8,811.82","39,652.02"
Sep-2024,"29,458.33",692.17,"4,454.55","10,857.27","45,462.32"
Oct-2024,"35,983.33",832.17,"5,272.73","12,902.73","54,990.96"


### **Calculate Credit Life Insurance On New Disbursement**

In [21]:
credit_insurance_fee_for_all_new_disbursements_df = other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        months_to_forecast=months_to_forecast,
    )

credit_insurance_fee_for_all_new_disbursements_df

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2024,"1,200.00",96.92,87.27,174.55,"1,558.74"
Feb-2024,"2,480.00",143.64,196.36,392.73,"3,212.73"
Mar-2024,"3,920.00",190.35,327.27,706.91,"5,144.53"
Apr-2024,"5,060.00",249.68,472.73,"1,099.64","6,882.05"
May-2024,"6,770.00",362.18,647.27,"1,570.91","9,350.36"
Jun-2024,"8,570.00",393.98,821.82,"2,101.09","11,886.89"
Jul-2024,"9,370.00",463.09,"1,127.27","2,788.36","13,748.72"
Aug-2024,"10,656.67",477.84,"1,454.55","3,524.73","16,113.78"
Sep-2024,"11,783.33",588.34,"1,781.82","4,342.91","18,496.40"
Oct-2024,"14,393.33",707.34,"2,109.09","5,161.09","22,370.86"


### **Calculate For Credit Loss For New Disbursements**

In [22]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, disbursement_parameters=disbursement_parameters
    )
)

provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2024,"5,400.00",0.00,960.00,960.00,"7,320.00"
Feb-2024,"5,760.00",0.00,"1,200.00","1,200.00","8,160.00"
Mar-2024,"4,320.00",0.00,"1,440.00","1,728.00","7,488.00"
Apr-2024,"3,420.00",0.00,"1,600.00","1,728.00","6,748.00"
May-2024,"5,130.00",0.00,"1,920.00","2,073.60","9,123.60"
Jun-2024,"5,400.00",0.00,"1,920.00","2,332.80","9,652.80"
Jul-2024,"6,000.00",0.00,"3,360.00","1,890.00","11,250.00"
Aug-2024,"7,700.00",0.00,"3,600.00","1,620.00","12,920.00"
Sep-2024,"7,700.00",0.00,"3,600.00",0.00,"11,300.00"
Oct-2024,"11,250.00",0.00,"3,600.00",0.00,"14,850.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [23]:
existing_loans = helper.columns_to_snake_case(existing_loans)
existing_loans = existing_loans.loc[existing_loans["closing_balance"] > 0]

In [24]:
existing_loans_schedules = borrowings.calculate_reducing_balance_loans_schedules(
    interest_rates=existing_loans["interest_rate"],
    effective_dates=existing_loans["disbursement_date"],
    frequencies=(existing_loans['interest_rate']*0+12),
    loan_identifiers=existing_loans["loan_number"],
    tenures=existing_loans["loan_term"],
    amounts=existing_loans["loan_amount"],
    is_interest_rate_annual=False
)

existing_loans_schedules.keys()

dict_keys(['outstanding_balance_at_start', 'capital_repayments', 'interest_payments', 'repayments'])

In [25]:
existing_loans_schedules['outstanding_balance_at_start']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"19,352.00","18,837.60","18,321.91","17,804.94","17,286.67","16,767.11","16,246.25","15,724.08","15,200.61","14,675.84","14,149.75","13,622.34","13,093.62","12,563.57","12,032.20","11,499.50","10,965.47","10,430.11","9,893.40","9,355.36","8,815.96","8,275.22","7,733.13","7,189.69","6,644.88","6,098.71","5,551.18","5,002.28","4,452.01","3,900.36","3,347.33","2,792.92","2,237.12","1,679.93","1,121.35",561.38
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,800.00,667.50,534.66,401.50,268.00,134.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [26]:
existing_loans_schedules['interest_payments']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,48.38,47.09,45.80,44.51,43.22,41.92,40.62,39.31,38.00,36.69,35.37,34.06,32.73,31.41,30.08,28.75,27.41,26.08,24.73,23.39,22.04,20.69,19.33,17.97,16.61,15.25,13.88,12.51,11.13,9.75,8.37,6.98,5.59,4.20,2.80,1.40
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,1.67,1.34,1.00,0.67,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"100,120,708,000,643",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,26.30,24.28,22.08,19.68,17.07,14.21,11.11,7.72,4.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,708,000,697",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.12,8.69,6.04,3.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [27]:
existing_loans_schedules['capital_repayments']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,514.40,515.69,516.97,518.27,519.56,520.86,522.16,523.47,524.78,526.09,527.41,528.72,530.05,531.37,532.70,534.03,535.37,536.70,538.05,539.39,540.74,542.09,543.45,544.81,546.17,547.53,548.90,550.27,551.65,553.03,554.41,555.80,557.19,558.58,559.98,561.38
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,132.50,132.83,133.17,133.50,133.83,134.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"100,120,708,000,643",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.45,24.47,26.67,29.07,31.68,34.54,37.64,41.03,44.73,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,708,000,697",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

### **Capital Repayment for Existing Loans**

In [28]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans.loc[helper.generate_columns(start_date=start_date, period=12)]

Jan-2024   2,973.78
Feb-2024   2,659.76
Mar-2024   2,630.20
Apr-2024   1,473.53
May-2024   1,477.22
Jun-2024   1,480.91
Jul-2024   1,484.61
Aug-2024   1,488.32
Sep-2024   1,492.04
Oct-2024   1,495.77
Nov-2024   1,499.51
Dec-2024   1,503.26
dtype: float64

### **Intererest Income For Existing Loans**

In [29]:
interest_income_existing_loans = existing_loans_schedules['interest_payments'].sum()
interest_income_existing_loans.loc[helper.generate_columns(start_date,12)]

Jan-2024   103.96
Feb-2024    65.31
Mar-2024    55.50
Apr-2024    48.93
May-2024    45.24
Jun-2024    41.55
Jul-2024    37.85
Aug-2024    34.14
Sep-2024    30.42
Oct-2024    26.69
Nov-2024    22.95
Dec-2024    19.20
dtype: float64

### **Calculate Other Income Existing Loans**

In [30]:
existing_loans = helper.columns_to_snake_case(existing_loans)
existing_loans = existing_loans.loc[existing_loans["closing_balance"] > 0]

In [31]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(
    existing_loans=existing_loans,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
other_income_existing_loans_df.head()

c:\Users\HP\Desktop\Claxon\Budgeting\manage-tenants-template\application\modeling\borrowings.py:53: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd.date_range(


,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Apr-2021,"1,317.83","4,937.63","6,255.46"
May-2021,"1,435.30","5,356.47","6,791.77"
Jun-2021,"1,651.48","6,142.33","7,793.81"
Jul-2021,"1,720.58","6,336.88","8,057.45"
Aug-2021,"1,838.08","6,455.25","8,293.34"


## **Interest Income**

In [32]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    start_date= start_date, 
    months_to_forecast= months_to_forecast
)
total_interest_income

Jan-2024       103.96
Feb-2024    47,367.84
Mar-2024    94,608.46
Apr-2024   145,689.21
May-2024   184,632.45
Jun-2024   237,005.94
Jul-2024   283,203.66
Aug-2024   336,245.92
Sep-2024   401,692.86
Oct-2024   461,722.39
Nov-2024   545,806.05
Dec-2024   614,960.94
Jan-2025   672,194.75
Feb-2025   717,933.01
Mar-2025   753,152.78
Apr-2025   781,404.29
May-2025   802,522.33
Jun-2025   817,628.58
Jul-2025   826,187.34
Aug-2025   827,602.93
Sep-2025   828,106.65
Oct-2025   828,106.65
Nov-2025   828,106.65
Dec-2025   828,106.65
Jan-2026   828,106.65
Feb-2026   828,106.65
Mar-2026   828,106.65
Apr-2026   828,106.65
May-2026   828,106.65
Jun-2026   828,106.65
Jul-2026   828,106.65
Aug-2026   828,106.65
Sep-2026   828,106.65
Oct-2026   828,106.65
Nov-2026   828,106.65
Dec-2026   828,106.65
Jan-2027   828,106.65
Feb-2027   828,106.65
Mar-2027   828,106.65
Apr-2027   828,106.65
May-2027   828,106.65
Jun-2027   828,106.65
Jul-2027   828,106.65
Aug-2027   828,106.65
Sep-2027   828,106.65
Oct-2027  

## **Other Income**

In [33]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements_df, 
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2024,"8,113.88","3,295.07","11,408.95"
Feb-2024,"16,171.43","6,578.96","22,750.39"
Mar-2024,"25,479.35","10,385.66","35,865.01"
Apr-2024,"33,796.58","13,794.24","47,590.83"
May-2024,"45,822.17","18,725.44","64,547.61"
Jun-2024,"58,420.65","23,798.50","82,219.15"
Jul-2024,"67,546.88","27,522.17","95,069.05"
Aug-2024,"79,333.12","32,252.29","111,585.40"
Sep-2024,"90,953.72","37,017.53","127,971.25"
Oct-2024,"110,011.00","44,766.44","154,777.43"


## **Depreciation**

In [34]:
details_of_assets = helper.columns_to_snake_case(details_of_assets)
details_of_assets.head()

,asset_id,acquisition_date,life,book_value,net_value,salvage_value,method,depreciation
0,FF-HRE-0024,01/09/2015,10,380.00,75.96,0,straight_line,NaN
1,FF-HRE-0025,01/09/2015,10,370.00,74.04,0,straight_line,NaN
2,FF-HRE-0026,01/09/2015,10,370.00,74.04,0,straight_line,NaN
3,FF-HRE-0027,01/09/2015,10,630.00,126.00,0,straight_line,NaN
4,FF-HRE-0028,01/09/2015,10,630.00,126.00,0,straight_line,NaN


In [35]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_assets=details_of_assets,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

In [36]:
depreciations_and_nbvs['dpns']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2024,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,0.00,0.00,82.11,46.70,35.47,14.59,27.92,26.39,28.17,20.50,18.33,22.17,22.17,22.17,22.17,22.17,22.17,22.17,36.33,10.98,2.77,2.77,2.77,2.77,2.77,2.77,2.77,2.77,32.93,21.99,39.34,13.02,13.02,3.38,11.92,0.00,3.58,0.00,49.34,78.53,2.63,3.92,1.32,3.69,78.53,0.70,0.70,0.70,3.15,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.54,19.61,10.50,3.14,3.14,3.14,2.35,2.35,2.35,2.35,2.35,2.35,2.35,2.35,1.48,1.48,1.48,1.48,1.48,1.48,1.32,1.32,1.32,1.32,2.02,2.02,3.17,3.08,3.08,5.25,5.25,5.25,5.25,0.92,45.98,22.08,16.33,19.25,23.67,4.00,16.25,2.75,11.08,7.67,77.34,4.92,4.92,1.48,1.48,2.49,2.04,1.00,1.00,13.04,8.79,22.83,1.91,3.81,3.66,2.53,104.55,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,3.66,6.17,5.62,4.48,1.75,4.83,1.50,13.00,5.96,8.67,30.33,5.20,5.96,8.67,34.67,27.08,13.00,5.42,21.12,3.79,3.79,28.17,30.33,11.48,17.33,39.00,10.29,22.25,2.08,2.96,20.79,11.95,5.00,6.49,53.00,10.08,8.08,43.60,10.81,15.23,9.06,10.19,27.26,9.61,10.62,0.79,74.60,51.24,51.

In [37]:
depreciations_and_nbvs['nbvs']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2024,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,0.00,0.00,492.68,420.34,567.47,248.00,558.33,448.60,"1,126.66",820.00,751.67,931.00,931.00,931.00,931.00,931.00,931.00,931.00,"1,889.22",614.99,155.40,155.40,155.40,155.40,155.40,155.40,155.40,155.40,"1,679.40","1,143.47","2,242.66",143.25,143.25,40.54,154.93,0.00,50.08,0.00,690.80,"2,669.98",71.15,168.34,93.40,107.12,"2,669.98",21.86,22.56,21.86,97.81,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,47.90,607.75,640.50,72.16,72.16,72.16,54.17,54.17,54.17,54.17,54.17,54.17,54.17,54.17,34.16,34.16,34.16,34.16,34.16,34.16,30.31,30.31,30.32,30.31,46.34,46.34,72.79,70.96,70.96,120.75,120.75,120.75,120.75,21.04,"1,082.39",529.82,392.04,462.00,567.95,96.00,390.00,66.00,266.04,183.95,"1,993.42",127.78,127.78,40.10,40.10,79.84,65.44,32.00,32.00,469.62,378.17,"1,004.74",80.00,160.14,153.93,106.16,"4,495.87",80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,80.00,153.93,265.09,241.88,219.52,86.00,217.58,67.50

## **Salaries**

In [57]:
importlib.reload(expenses)

<module 'application.modeling.expenses' from 'c:\\Users\\HP\\Desktop\\Claxon\\Budgeting\\manage-tenants-template\\application\\modeling\\expenses.py'>

In [58]:
salaries_and_pension_and_statutory_contributions_df = (
    expenses.calculate_salaries_and_pension_and_statutory_contributions(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        other_parameters = other_parameters, 
        months_to_forecast=months_to_forecast,
        start_date=start_date,
    )
)
salaries_and_pension_and_statutory_contributions_df

,agent_commission,credit_officer_salaries,credit_officer_commission,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2024,"7,200.00",600.00,"3,600.00","17,000.00","28,400.00","1,999.36"
Feb-2024,"9,000.00",600.00,"3,840.00","17,000.00","30,440.00","1,999.36"
Mar-2024,"12,240.00",600.00,"4,320.00","17,000.00","34,160.00","1,999.36"
Apr-2024,"14,800.00",600.00,"3,420.00","17,000.00","35,820.00","1,999.36"
May-2024,"17,760.00",900.00,"5,130.00","17,000.00","40,790.00","2,033.44"
Jun-2024,"19,380.00",900.00,"5,400.00","17,000.00","42,680.00","2,033.44"
Jul-2024,"27,300.00","1,200.00","6,000.00","17,000.00","51,500.00","2,067.52"
Aug-2024,"29,250.00","1,400.00","7,700.00","17,000.00","55,350.00","2,090.24"
Sep-2024,"31,500.00","1,400.00","7,700.00","17,000.00","57,600.00","2,090.24"
Oct-2024,"31,500.00","1,800.00","11,250.00","17,000.00","61,550.00","2,135.68"


## **Provisions**


In [59]:
provision_for_credit_loss_for_all_new_disbursements_df = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
    )
)
provision_for_credit_loss_for_all_new_disbursements_df

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2024,"5,400.00",0.00,960.00,960.00,"7,320.00"
Feb-2024,"5,760.00",0.00,"1,200.00","1,200.00","8,160.00"
Mar-2024,"4,320.00",0.00,"1,440.00","1,728.00","7,488.00"
Apr-2024,"3,420.00",0.00,"1,600.00","1,728.00","6,748.00"
May-2024,"5,130.00",0.00,"1,920.00","2,073.60","9,123.60"
Jun-2024,"5,400.00",0.00,"1,920.00","2,332.80","9,652.80"
Jul-2024,"6,000.00",0.00,"3,360.00","1,890.00","11,250.00"
Aug-2024,"7,700.00",0.00,"3,600.00","1,620.00","12,920.00"
Sep-2024,"7,700.00",0.00,"3,600.00",0.00,"11,300.00"
Oct-2024,"11,250.00",0.00,"3,600.00",0.00,"14,850.00"


## **Calculating Finance Costs**

In [60]:
details_of_short_term_borrowings = helper.columns_to_snake_case(
    details_of_short_term_borrowing
)
details_of_long_term_borrowings = helper.columns_to_snake_case(
    details_of_long_term_borrowing
)

In [61]:
long_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_long_term_borrowings)
long_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [62]:
short_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_short_term_borrowings)
short_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [63]:
capital_repayment_borrowings_df = pd.concat(
    [
        long_term_borrowings_schedules["capital_repayments"],
        short_term_borrowings_schedules["capital_repayments"],
    ],
).fillna(0)

capital_repayment_borrowings_df.loc['total'] = capital_repayment_borrowings_df.sum()

In [64]:
finance_costs_df = pd.concat(
    [
        long_term_borrowings_schedules["interest_payments"],
        short_term_borrowings_schedules["interest_payments"],
    ],
).fillna(0)


finance_costs_df.loc["total"] = finance_costs_df.sum()
finance_costs_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,"1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZLA-Cargo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"12,457.50",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,000.00",0.00,0.00,0.00,0.00,0.00,"2,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,784.00",0.00,0.00,0.00,0.00,0.00,"2,784.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BERAMASAMBA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,900.00,0.00,0.00,0.00,0.00,0.00,900.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,832.41","1,806.04","1,779.54","1,752.91","1,726.16","1,699.27","1,672.25","1,645.10","1,617.82","1,590.40","1,562.85","1,535.17","1,507.35","1,479.40","1,451.31","1,423.08","1,394.72","1,366.22","1,337.58","1,308.80","1,279.88","1,250.82","1,221.61","1,192.27","1,162.78","1,133.15","1,103.38","1,073.46","1,043.39","1,013.18",982.82,952.32,921.66,890.86,859.90,828.80,797.54,766.13,734.57,702.86,670.99,638.96,606.78,574.45,541.95,509.30,476.49,443.52,410.39,377.09,343.64,310.02,276.24,242.29,208.18,173.90,139.46,104.85,70.0

## **Income Statement**

In [65]:
income_statement_df = income_statement.generate_income_statement_template(
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
income_statement_df

,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027,Jan-2028,Feb-2028,Mar-2028,Apr-2028,May-2028,Jun-2028,Jul-2028,Aug-2028,Sep-2028,Oct-2028,Nov-2028,Dec-2028
INCOME_STATEMENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Variable Expenses**

In [ ]:
uncertain_expenses = expenses.calculate_uncertain_expenses(
    expenses_uncertain=expenses_uncertain,
    other_parameters=other_parameters,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
uncertain_expenses

### **Calculate Change In Provision For Credit Loss**

In [ ]:
change_in_provision_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        start_date=start_date, 
        months_to_forecast=months_to_forecast
    )
)

change_in_provision_for_credit_loss

### **Calculate Business Acquisition**

In [ ]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=disbursement_parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=disbursement_parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

### **Insert Revenue**

In [ ]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)

income_statement_df

### **Insert Static And Variable Inputs**

In [ ]:
income_statement_df = income_statement.insert_expenses(
    income_statement=income_statement_df,
    expenses_certain=expenses_certain,
    uncertain_expenses=uncertain_expenses,
)

income_statement_df

### **Insert Salaries and Pensions And Statutory Contributions**

In [ ]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df=salaries_and_pension_and_statutory_contributions_df,
)

income_statement_df

### **Insert Depreciation**

In [ ]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df,
    depreciation=depreciations_and_nbvs["dpns"]["total"],
)

### **Insert Change in Provision For Credit Loss**

In [ ]:
income_statement_df = income_statement.insert_credit_loss_provision(
    income_statement=income_statement_df, 
    change_in_provision_for_credit_loss=change_in_provision_for_credit_loss,
)

In [ ]:
provision_for_credit_loss = provision_for_credit_loss_for_all_new_disbursements["total"]
provision_for_credit_loss

### **Insert Business Acquisition**

In [ ]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition
)

### **Aggregations**

In [ ]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [ ]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)

### **Insert Finance Cost**

In [ ]:
income_statement_df.loc["Finance Costs"] = finance_costs_df.loc["total"]
income_statement_df.loc["Finance Costs"] = income_statement_df.loc[
    "Finance Costs"
].fillna(0)
income_statement_df

### **Aggregate Finance Cost**

In [ ]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculate Profit Before Tax**

In [ ]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)

### **Calculate Tax**

In [ ]:
income_statement_df = income_statement.calculate_tax(
    income_statement=income_statement_df, tax_rate=other_parameters.loc["TAX_RATE"]
)
income_statement_df

## **Direct Cashflow**

In [ ]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    start_date=start_date, months_to_forecast=months_to_forecast
)

direct_cashflow_df

### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [ ]:
direct_cashflow_df.loc[
    "Receipts From Receivables"
] = helper.change_period_index_to_strftime(
    helper.add_series(
        [
            other_parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
            other_parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
            other_parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
        ]
    )
)

direct_cashflow_df.loc[
    "Purchase Of Inventory"
] = helper.change_period_index_to_strftime(other_parameters.loc["NEW_INVENTORY"])
direct_cashflow_df.loc[
    "Payments To Payables"
] = -helper.change_period_index_to_strftime(
    helper.add_series(
        [
            other_parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
            other_parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
        ]
    )
)

direct_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    other_parameters.loc["DIVIDEND_PAID"]
)

#### **From Calculations/Income Statement**

In [ ]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = -income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = -helper.change_period_index_to_strftime(new_disbursements_df['total'])

In [ ]:
direct_cashflow_df

### **Equity and Intercompany Loans**

In [ ]:
direct_cashflow_df = direct_cashflow.add_equity_and_intercompany_loans(
    other_parameters=other_parameters, direct_cashflow_df=direct_cashflow_df
)

### **Other Assets**


In [ ]:
direct_cashflow_df = direct_cashflow.add_other_assets(
    other_parameters=other_parameters, direct_cashflow_df=direct_cashflow_df
)

### **Tax Paid**

In [ ]:
tax_schedule = direct_cashflow.generate_tax_schedule(
    taxation=income_statement_df.loc["Taxation"],
    opening_balance=opening_balances["DEFERED_TAXATION"].iat[0],
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

tax_schedule

In [ ]:
direct_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']

### **Calculate Operating Expenses**

In [ ]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

### **Insert Operating Expenses**

In [ ]:
direct_cashflow_df.loc['Operating Expenses']  = -operating_expenses
direct_cashflow_df


### **Calculate Capital Expenses**

In [ ]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_assets=details_of_assets,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

capital_expenses

### **Insert Capital Expenses**

In [ ]:
direct_cashflow_df.loc['Capital Expenses'] = -capital_expenses

### **Calculate Direct Cashflow Borrowings**

In [ ]:
long_and_short_term_borrowing_df = (
    direct_cashflow.calculate_long_and_short_term_borrowing_for_direct_cashflow(
        details_of_long_term_borrowing=details_of_long_term_borrowing,
        details_of_short_term_borrowing=details_of_short_term_borrowing,
        start_date=start_date,
        months_to_forecast=months_to_forecast,
    )
)

long_and_short_term_borrowing_df

### **Insert Direct Cashflow Borrowing**

In [ ]:
direct_cashflow_df.loc['Short Term Borrowing'] = long_and_short_term_borrowing_df['short_term_borrowing']
direct_cashflow_df.loc['Long Term Borrowing'] = long_and_short_term_borrowing_df['long_term_borrowing']

In [ ]:
direct_cashflow_df

### **Calculate Capital Repayments**

In [ ]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayments'].sum()
    , capital_repayment_new_disbursements_df['total']]
)

capital_repayment.loc[helper.generate_columns(start_date,months_to_forecast)]

### **Insert Capital Repayment**

In [ ]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [ ]:
capital_repayment_borrowings_df

### **Insert Capital Repayment on New Borrowings**

In [ ]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]

direct_cashflow_df.loc["Capital Repayment On Borrowings"] = direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
].fillna(0)

### **Calculate Direct Cashflow Totals**

In [ ]:
direct_cashflow_df

In [ ]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH INFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Inflows")
].sum()


direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH OUTFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Outflows")
].sum()

direct_cashflow_df.loc["Net Increase/Decrease In Cash"] = (
    direct_cashflow_df.loc["Total Cash Inflows"]
    + direct_cashflow_df.loc["Total Cash Outflows"]
)

### **Calculate Opening and Closing Balances**

In [ ]:
depreciations_and_nbvs['dpns'].sum(axis = 1)

In [ ]:
depreciations_and_nbvs['nbvs'].sum(axis = 1)

In [ ]:
direct_cashflow_df

In [ ]:
tax_schedule

In [ ]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)

direct_cashflow_df

### **Calculate imtt (2% Tax)**

In [ ]:
income_statement_df.loc['2% Taxation']  = -direct_cashflow_df.loc['Total Cash Outflows']*imtt

## **Calculate Profit and Loss for the Period**


In [ ]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

## **Loan Book**

In [ ]:
loan_book_df = loan_book.generate_loan_book_template(
    start_date=start_date, months_to_forecast=months_to_forecast
)
loan_book_df

### **Insert Loan Book Items**

In [ ]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans.loc[helper.generate_columns(start_date,months_to_forecast)]

In [ ]:
total_capital_repayments = loan_book.aggregate_new_and_existing_loans_capital_repayments(
    capital_repayments_new_disbursements_df = capital_repayment_new_disbursements_df,
    capital_repayments_existing_loans = capital_repayment_existing_loans,
    start_date = start_date,
    months_to_forecast = months_to_forecast,
)

total_capital_repayments

In [ ]:
# epening_balance_on_loan_book = existing_loans.loc[existing_loans['closing_balance']>0]['closing_balance'].sum()

loan_book_df = loan_book.insert_loan_book_items(
    loan_book=loan_book_df,
    opening_balance_on_loan_book=float(opening_balances["LOAN_BOOK"].iat[0]),
    total_interest_income=total_interest_income,
    total_capital_repayments=total_capital_repayments,
    disbursements=helper.change_period_index_to_strftime(new_disbursements_df["total"]),
)

loan_book_df

### **Calculate Opening and Closing Balances for Loan Book**

In [ ]:
loan_book_df = helper.calculate_opening_and_closing_balances(loan_book_df)
loan_book_df

## **Balance Sheet**


In [ ]:
balance_sheet_df = balance_sheet.generate_balance_sheet_template(
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

balance_sheet_df

### **Insert Available Balance Sheet Items**

In [ ]:
depreciations_and_nbvs.keys()

In [ ]:
balance_sheet_df.loc["Property Plant And Equipment"] = depreciations_and_nbvs["nbvs"][
    "total"
]

balance_sheet_df.loc["Loan Book"] = loan_book_df.loc["Closing Balance"]

balance_sheet_df.loc["Cash On Hand"] = direct_cashflow_df.loc["Closing Balance"]

balance_sheet_df.loc[
    "Provisions"
] = provision_for_credit_loss_for_all_new_disbursements["total"]


balance_sheet_df.loc["Provision For Taxation"] = helper.change_period_index_to_strftime(
    other_parameters.loc["PROVISION_FOR_TAX"]
)

balance_sheet_df

### **Generate Borrowings Schedules**

In [ ]:
short_term_loans_schedules = balance_sheet.calculate_short_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=short_term_borrowings_schedules['capital_repayments'].sum(),
    opening_balances=opening_balances,
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)

short_term_loans_schedules

In [ ]:
capital_repayment_borrowings_df

In [ ]:
long_and_short_term_borrowing_df

In [ ]:
long_term_loans_schedules = balance_sheet.calculate_long_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=long_term_borrowings_schedules['capital_repayments'].sum(),
    opening_balances=opening_balances,
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)

long_term_loans_schedules

### **Insert Borrowings and Loans**


In [ ]:
balance_sheet_df.loc["Loans"] = long_term_loans_schedules.loc["Closing Balance"]
balance_sheet_df.loc["Borrowings"] = short_term_loans_schedules.loc["Closing Balance"]

### **Generate Receivables and Payables Schedules**

In [ ]:
trade_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["TRADE_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_TRADE_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

other_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["OTHER_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_OTHER_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

intergroup_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["INTERGROUP_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_INTERGROUP_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

trade_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["TRADE_PAYABLES"].iat[0],
    payments_to_payables=other_parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
    new_payables=other_parameters.loc["NEW_TRADE_PAYABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

other_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["OTHER_PAYABLES"].iat[0],
    payments_to_payables=other_parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
    new_payables=other_parameters.loc["NEW_OTHER_PAYABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)


balance_sheet_df.loc["Trade Payables"] = trade_payables_schedule.loc["Closing Balance"]
balance_sheet_df.loc["Other Payables"] = other_payables_schedule.loc["Closing Balance"]

balance_sheet_df.loc["Trade Receivables"] = trade_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Other Receivables"] = other_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Intergroup Receivables"] = intergroup_receivables_schedule.loc[
    "Closing Balance"
]


balance_sheet_df

In [ ]:
balance_sheet_df.loc['Deferred Taxation'] = tax_schedule.loc['Closing Balance']

In [ ]:
inventories_schedule = balance_sheet.generate_inventories_schedule(
    opening_inventories=opening_balances["INVENTORIES"].iat[0],
    new_inventories=other_parameters.loc["NEW_INVENTORY"],
    inventories_used=other_parameters.loc["INVENTORY_USED"],
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

inventories_schedule

In [ ]:
balance_sheet_df.loc['Inventories']  = inventories_schedule.loc['Closing Balance']

### **Equity and Intercompany Loans**


In [ ]:
balance_sheet_df.loc["Issued Share Capital"] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHARE_CAPITAL"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)

balance_sheet_df.loc["Intercompany Loans"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTERCOMPANY_LOANS"].cumsum()
    + opening_balances["INTERCOMPANY_LOANS"].iat[0]
)


balance_sheet_df.loc["Share Premium"] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHARE_PREMIUM"].cumsum() + opening_balances["SHARE_PREMIUM"].iat[0]
)

balance_sheet_df.loc[
    "Other Components Of Equity"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["OTHER_COMPONENTS_OF_EQUITY"].cumsum()
    + opening_balances["OTHER_COMPONENTS_OF_EQUITY"].iat[0]
)

balance_sheet_df.loc["Treasury Shares"] = helper.change_period_index_to_strftime(
    other_parameters.loc["TREASURY_SHARES"].cumsum()
    + opening_balances["TREASURY_SHARES"].iat[0]
)

balance_sheet_df.loc["Retained Earnings"] = (
    income_statement_df.loc["PROFIT/(LOSS) FOR PERIOD"]
    - helper.change_period_index_to_strftime(other_parameters.loc["DIVIDEND_PAID"])
).cumsum()

balance_sheet_df.loc["Capital And Reserves"] = balance_sheet_df.loc[
    "Issued Share Capital":"Retained Earnings"
].sum()

### **Other Assets**

In [ ]:
balance_sheet_df.loc["Intangible Assets"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTANGIBLE_ASSETS"].cumsum()
    + opening_balances["INTANGIBLE_ASSETS"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Subsidiaries"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_IN_SUBSIDIARIES"].cumsum()
    + opening_balances["INVESTMENT_IN_SUBSIDIARIES"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Associates"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_IN_ASSOCIATES"].cumsum()
    + opening_balances["INVESTMENT_IN_ASSOCIATES"].iat[0]
)

balance_sheet_df.loc["Investment Properties"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_PROPERTIES"].cumsum()
    + opening_balances["INVESTMENT_PROPERTIES"].iat[0]
)

balance_sheet_df.loc["Equity Investments"] = helper.change_period_index_to_strftime(
    other_parameters.loc["EQUITY_INVESTMENTS"].cumsum()
    + opening_balances["EQUITY_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Long Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["LONG_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["LONG_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Short Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Loans To Related Entities"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["LOANS_TO_RELATED_ENTITIES"].cumsum()
    + opening_balances["LOANS_TO_RELATED_ENTITIES"].iat[0]
)

In [ ]:
balance_sheet_df

### **Calculate Totals**


In [ ]:
balance_sheet_df = balance_sheet.sum_financial_statements_totals(balance_sheet_df)
balance_sheet_df = balance_sheet.calculate_final_balances(balance_sheet_df=balance_sheet_df)
balance_sheet_df

## **Indirect Cashflow**

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.generate_statement_of_cashflow_template(
        start_date, months_to_forecast
    )
)

### **Insert Available Balances**

In [ ]:
statement_of_cashflow_df.loc["Profit/(loss) Per I/S"] = income_statement_df.loc[
    "PROFIT/(LOSS) FOR PERIOD"
]
statement_of_cashflow_df.loc["Depreciation"] = income_statement_df.loc["Depreciation"]
statement_of_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    other_parameters.loc["DIVIDEND_PAID"]
)
statement_of_cashflow_df.loc[
    "Treasury Movements"
] = helper.change_period_index_to_strftime(other_parameters.loc["TREASURY_MOVEMENTS"])
statement_of_cashflow_df.loc[
    "Interest Expense Accrued"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTEREST_EXPENSE_ACCRUED"]
)
statement_of_cashflow_df.loc[
    "Other Non-Cash Items"
] = helper.change_period_index_to_strftime(other_parameters.loc["OTHER_NON_CASH_ITEMS"])
statement_of_cashflow_df.loc["Interest Paid"] = -finance_costs_df.loc["total"]
statement_of_cashflow_df.loc["Tax Paid"] = tax_schedule.loc["Tax Paid"]
statement_of_cashflow_df.loc[
    "Repayment Of Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]
statement_of_cashflow_df.loc["Purchase Of Fixed Assets"] = capital_expenses
statement_of_cashflow_df.loc["Increase/(Decrease) In Short Term Borrowings"] = (
    short_term_loans_schedules.loc["Closing Balance"]
    - short_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df.loc["Increase/(Decrease) In Long Term Borrowings"] = (
    long_term_loans_schedules.loc["Closing Balance"]
    - long_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df

In [ ]:
statement_of_cashflow_df.loc[
    "Cash From Operations Before WC"
] = statement_of_cashflow_df.iloc[
    1 : statement_of_cashflow_df.index.get_loc("Cash From Operations Before WC")
].sum()

In [ ]:

statement_of_cashflow_df

### **Working Capital Movements**

In [ ]:
change_in_receivables = (
    other_receivables_schedule.loc["Closing Balance"]
    - other_receivables_schedule.loc["Opening Balance"]
    + trade_receivables_schedule.loc["Closing Balance"]
    - trade_receivables_schedule.loc["Opening Balance"]
    + intergroup_receivables_schedule.loc["Closing Balance"]
    - intergroup_receivables_schedule.loc["Opening Balance"]
)

change_in_receivables

In [ ]:
change_in_payables = (
    trade_payables_schedule.loc["Closing Balance"]
    - trade_payables_schedule.loc["Opening Balance"]
    + other_payables_schedule.loc["Closing Balance"]
    - other_payables_schedule.loc["Opening Balance"]
)
change_in_payables

In [ ]:
change_in_loan_book_principle = loan_book_df.loc["New Disbursements":"Repayments"].sum()
change_in_loan_book_principle

In [ ]:
change_in_loan_book_interest = loan_book_df.loc['Interest Income']
change_in_loan_book_interest

In [ ]:
borrowings_schedule = long_term_loans_schedules + short_term_loans_schedules
change_in_borrowings = borrowings_schedule.loc['Closing Balance'] - borrowings_schedule.loc['Opening Balance']
change_in_borrowings

In [ ]:
statement_of_cashflow_df.loc['(Increase)/Decrease In Receivables'] = change_in_receivables
statement_of_cashflow_df.loc['Increase/(Decrease) In Payables'] = change_in_payables
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Principle)'] = change_in_loan_book_principle
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Interest)'] = change_in_loan_book_interest
statement_of_cashflow_df.loc['Increase/(Decrease) In Borrowings'] = change_in_borrowings

In [ ]:
statement_of_cashflow_df.loc["Cash From Operations After WC"] = (
    statement_of_cashflow_df.loc["Cash From Operations Before WC"]
    + statement_of_cashflow_df.iloc[
        statement_of_cashflow_df.index.get_loc("Working Capital Movements")
        + 1 : statement_of_cashflow_df.index.get_loc("Cash From Operations After WC")
    ].sum()
)

In [ ]:
statement_of_cashflow_df.loc[
    "Net Cash Flow From Operations"
] = statement_of_cashflow_df.loc["Cash From Operations After WC":"Tax Paid"].sum()

In [ ]:
statement_of_cashflow_df

### **Calculating Totals**

In [ ]:
statement_of_cashflow_df = balance_sheet.sum_financial_statements_totals(statement_of_cashflow_df)


statement_of_cashflow_df.loc['Net Increase/(Decrease) In Cash'] = (
    statement_of_cashflow_df.loc["Net Cash Flow From Operations"]
    + statement_of_cashflow_df.loc["Cash Flow From Investing Activities"]
    + statement_of_cashflow_df.loc["Cash Flow From Financing Activities"]
)
statement_of_cashflow_df

### **Calculating Cash at End and Beginning of Period**

In [ ]:
statement_of_cashflow_df = (
    statement_of_cashflows.calculate_cash_at_end_and_beginning_of_period(
        statement_of_cashflow_df=statement_of_cashflow_df,
        opening_balances=opening_balances,
    )
)

statement_of_cashflow_df

In [ ]:
income_statement_df

In [ ]:
balance_sheet_df